## 1.Closed-form $P_t$ function

In [19]:
import numpy as np

def logistic_exact(t_values, r, K, P0):
    """
    Evaluate the closed-form logistic solution P(t) at given times.

    Parameters
    ----------
    t_values : float or array-like
        Time(s) where P(t) is evaluated.
    r : float
        Growth rate (r > 0).
    K : float
        Carrying capacity (K > 0).
    P0 : float
        Initial population (0 < P0 < K).
    Returns
    -------
    P : numpy.ndarray or float
        P(t) values matching the shape/type of t_values.
    """
    t = np.asarray(t_values, dtype=float)
    A = (K - P0) / P0
    P = K / (1.0 + A * np.exp(-r * t))
    return float(P) if np.isscalar(t_values) else P


Example

In [20]:
r, K, P0 = 0.5, 100.0, 10.0
t_list = [0, 1, 2, 5, 10, 20]
print("output of vector t:", logistic_exact(t_list, r, K, P0))

t=1
print("output of scalar t:", logistic_exact(t, r, K, P0))

output of vector t: [10.         15.4828099  23.19693167 57.51208514 94.28256186 99.95915675]
output of scalar t: 15.482809896025467


## 2.Forward Euler method for $P_t$

In [21]:
import numpy as np

def forward_euler_logistic(r, K, P0, t0, tf, dt):
    """
    Forward Euler approximation for logistic growth.

    Returns
    -------
    t : numpy.ndarray
        Time grid from t0 to tf (inclusive).
    P : numpy.ndarray
        Euler approximation values.
    """
    if dt <= 0:
        raise ValueError("dt must be positive")
    n_steps = int(np.ceil((tf - t0) / dt))
    t = t0 + dt * np.arange(n_steps + 1)
    t[-1] = tf  # force exact endpoint (optional)

    P = np.zeros_like(t, dtype=float)
    P[0] = P0

    for n in range(len(t) - 1):
        h = t[n+1] - t[n]  # actual step (last step may differ slightly)
        P[n+1] = P[n] + h * (r * P[n] * (1.0 - P[n] / K))

    return t, P


Closed-form solution of $t$ when $P_t = K_t$


In [22]:
import math

def t_half_exact(r, K, P0):
    return (1.0 / r) * math.log((K - P0) / P0)

t_half_exact(0.5, 100.0, 10.0)

4.394449154672439

Numerical estimate from Euler of $t$ when $P_t = K_t$

In [23]:
import numpy as np

def t_half_from_euler(t, P, K):
    target = 0.5 * K
    idx = np.where(P >= target)[0]
    if len(idx) == 0:
        return None
    i = int(idx[0])
    if i == 0:
        return float(t[0])

    # linear interpolation between (t[i-1], P[i-1]) and (t[i], P[i])
    t0, t1 = float(t[i-1]), float(t[i])
    P0, P1 = float(P[i-1]), float(P[i])
    return t0 + (target - P0) * (t1 - t0) / (P1 - P0)

## 3.Compare Exact and Euler forward result when finding time hit half K:

In [24]:
r, K, P0 = 0.5, 100.0, 10.0
t0, tf = 0.0, 20.0
dt = 0.1

# Exact half-time
print("Exact t_half =", t_half_exact(r, K, P0))

# Euler solve + half-time
t_eu, P_eu = forward_euler_logistic(r, K, P0, t0, tf, dt)
print("Euler t_half (interp) =", t_half_from_euler(t_eu, P_eu, K))


Exact t_half = 4.394449154672439
Euler t_half (interp) = 4.445606302371804
